In [1]:
import pandas as pd
import numpy as np
import PyPDF2
import os
import re

reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-september-2023-pdf.pdf')

page4 = reader.pages[3]
page4_text = page4.extract_text()

print(page4_text)

startpoint = page3_text.find('Back-up Servicer Fees') + 21
startpoint

len('Back-up Servicer Fees')

endpoint = page3_text.find(" ", startpoint+1)

float(page3_text[startpoint:endpoint].replace(',','').strip())

len('Interest: \n')

midpoint = page3_text.find('\n', startpoint) + 2
midpoint

endpoint = page3_text.find(" ", midpoint+1)
endpoint

float(page3_text[midpoint:endpoint].replace(',','').strip())

ending_loan_bal = float(page.extract_text()[startpoint:endpoint].replace(',','').strip())

collat_dict = {}
collat_dict['ending_loan_balance'] = [ending_loan_bal]

startpoint = page.extract_text().find('roposed Loans, and Replacement Loans')

midpoint = page.extract_text().find('%', startpoint) + 3

endpoint = page.extract_text().find('%', midpoint+1) - 1

collat_dict['num_loans'] = [float(page.extract_text()[midpoint:endpoint].split()[0].replace(',','').strip())]
collat_dict['wac'] = [float(page.extract_text()[midpoint:endpoint].split()[-1].replace(',','').strip())]
collat_dict['fico'] = [int(page.extract_text()[endpoint+4:endpoint+7])]

startpoint = endpoint + 1

endpoint = page.extract_text().find("Page", startpoint)

collat_dict['wart'] = [float(page.extract_text()[startpoint+7:endpoint-1])]

def get_filename(filename):
    """
    returns name of onemain files
    filename should be a OS dir object"""
    return str(filename)[str(filename).find('\\')+1:str(filename).find(".", 1)].split()[1].replace("'", "")

def create_readers(directory):
    """for the given directory, returns a dict of PyPDF2 readers for each filename in the directory
    dict key is the name of the file
    dict value is the reader
    this function only looks for the naming convention used in  OneMain PDFs"""
    readers = {}
    for filename in os.scandir(directory):
        if filename.is_file():
            name = get_filename(filename)
            readers[name] = PyPDF2.PdfReader(filename.path)
    return readers

directory = './dataset'
collat_dict = {}
collat_dict['ending_loan_balance'] = []
collat_dict['num_loans'] = []
collat_dict['wac'] = []
collat_dict['fico'] = []
collat_dict['wart'] = []
collat_dict['total_prin_coll'] = []
collat_dict['total_int_coll'] = []

readers_dict = create_readers(directory)

dates_pattern = "[0-9]{2}[/][0-9]{2}[/][0-9]{4}"

for key, value in readers_dict.items():
    if 'monthly-servicer-report' in key: #looking at just the trustee reports 
        reader = value
        page2 = reader.pages[1] #second page for loan information
        page2_text = page2.extract_text()
        dates = re.findall(dates_pattern, page2_text)
        payment_date = dates[0]
        end_of_coll_period = dates[1]
        startpoint = page2.extract_text().find('Loan Action Date Loan Principal Balance') + 39
        endpoint = page2.extract_text().find('\n', startpoint) - 1
        ending_loan_bal = float(page2.extract_text()[startpoint:endpoint].replace(',','').strip())            
        collat_dict['ending_loan_balance'].append((payment_date, ending_loan_bal))
        startpoint = page2.extract_text().find('roposed Loans, and Replacement Loans')
        midpoint = page2.extract_text().find('%', startpoint) + 3
        endpoint = page2.extract_text().find('%', midpoint+1) - 1
        collat_dict['num_loans'].append((payment_date, float(page2.extract_text()[midpoint:endpoint].split()[0].replace(',','').strip())))
        collat_dict['wac'].append((payment_date, float(page2.extract_text()[midpoint:endpoint].split()[-1].replace(',','').strip())))
        collat_dict['fico'].append((payment_date, int(page2.extract_text()[endpoint+4:endpoint+7])))
        startpoint = endpoint + 1
        endpoint = page2.extract_text().find("Page", startpoint)
        collat_dict['wart'].append((payment_date, float(page2.extract_text()[startpoint+7:endpoint-1])))
        page3 = reader.pages[2]
        page3_text = page3.extract_text()
        startpoint = page3_text.find('Total Principal Collections') + 28
        midpoint = page3_text.find('\n', startpoint) + 2
        endpoint = page3_text.find(" ", midpoint+1)
        collat_dict['total_prin_coll'].append((payment_date, float(page3_text[midpoint:endpoint].replace(',','').strip())))
        startpoint = page3_text.find('Interest:') + 11
        endpoint = page3_text.find(" ", startpoint+1)
        collat_dict['total_int_coll'] = float(page3_text[startpoint:endpoint].replace(',','').strip())

collat_dict

create_readers('./dataset')


reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-august-2023-pdf.pdf')
print(filename.path)
page2 = reader.pages[1]
page2_text = page.extract_text()
print(page2_text)
reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-june-2023-pdf.pdf')
print(filename.path)
page2 = reader.pages[1]
page2_text = page.extract_text()
print(page2_text)

reader = PyPDF2.PdfReader('./dataset/omfit-2019-2-monthly-servicer-report-september-2023-pdf.pdf')
print(filename.path)
page2 = reader.pages[1]
page2_text = page.extract_text()
print(page2_text)

text = "Monthly Servicer Report Collection Period  3010/16/202309/30/2023 Transaction MonthPayment DateEnd DateBeginning Date  4909/01/2023 30/360 Days"

r = "[0-9]{2}[/][0-9]{2}[/][0-9]{4}"
re.findall(r, text)

payment_date = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='G', skiprows=6, nrows=1, header=None, names=['payment_date'])
payment_date = payment_date['payment_date'][0]
payment_date

pool_info_df = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=55, nrows=5, header=None, names=['end_of_period']).T.reset_index(drop=True)

pool_info_df.columns = ['wac', 'wa_fico', 'wart', 'ending_balance', 'num_loans']
pool_info_df.index = [payment_date]
pool_info_df

prin_coll = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=77, nrows=1, header=None, names=['total_prin_coll'])
prin_coll.index = [payment_date]
prin_coll

int_coll = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=82, nrows=1, header=None, names=['total_int_coll'])
int_coll.index = [payment_date]
int_coll

distributions = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=101, nrows=19, header=None, names=['distributions'])

wf_steps = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='A', skiprows=101, nrows=19, header=None, names=['wf_steps'])

distributions.index = [wf_steps['wf_steps']]

distributions = distributions.T
distributions.index = [payment_date]
distributions

rsv_acct = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=126, nrows=1, header=None, names=['reserve_acct_bal'])
rsv_acct.index = [payment_date]
rsv_acct

oc = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='F', skiprows=151, nrows=3, header=None, names=['oc'])
oc.index = ['total_oc', 'required_oc', 'in_compliance']
oc = oc.T
oc.index = [payment_date]
oc

events_df = pd.read_excel('.\dataset\omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx', usecols='A, C:F', skiprows=181, nrows=31, header=None, names=['event', 'initial_pool', 'current_pool', 'limit', 'in_compliance'])
events_df.dropna(axis=0, how='all', inplace=True)
events_df = events_df.reset_index(drop=True)

events_df = events_df[(events_df['event'] != 'AMORTIZATION EVENTS') & (pd.notna(events_df['event']))].reset_index(drop=True)
events_df

In [3]:
directory = "./dataset"
filenames = []
payment_dates = []
for filename in os.scandir(directory):
    if filename.is_file():
        filenames.append(filename)
        payment_date = pd.read_excel(filename, usecols='G', skiprows=6, nrows=1, header=None, names=['payment_date'])
        payment_date = payment_date['payment_date'][0]
        payment_dates.append(payment_date)

In [9]:
files_dates = list(zip(payment_dates, filenames))

In [10]:
files_dates = sorted(files_dates, key = lambda x: x[0])
files_dates

[(Timestamp('2019-10-15 00:00:00'), <DirEntry '2019-10-15-1001257227.xlsx'>),
 (Timestamp('2019-11-14 00:00:00'), <DirEntry '2019-11-14-1500125662.xlsx'>),
 (Timestamp('2019-12-16 00:00:00'), <DirEntry '2019-12-16-1001259387.xlsx'>),
 (Timestamp('2020-01-14 00:00:00'), <DirEntry '2020-01-14-1500126468.xlsx'>),
 (Timestamp('2020-02-14 00:00:00'), <DirEntry '2020-02-14-1001260774.xlsx'>),
 (Timestamp('2020-03-16 00:00:00'), <DirEntry '2020-03-16-1500128078.xlsx'>),
 (Timestamp('2020-04-14 00:00:00'), <DirEntry '2020-04-14-1500128822.xlsx'>),
 (Timestamp('2020-05-14 00:00:00'), <DirEntry '2020-05-14-1001263816.xlsx'>),
 (Timestamp('2020-06-15 00:00:00'), <DirEntry '2020-06-15-1001264428.xlsx'>),
 (Timestamp('2020-07-14 00:00:00'), <DirEntry '2020-07-14-1500130937.xlsx'>),
 (Timestamp('2020-08-14 00:00:00'),
  <DirEntry 'OMFIT-2019-2-Monthly-Servicer-Report-July-2020-Excel_InvestorVersion.xlsx'>),
 (Timestamp('2020-09-14 00:00:00'),
  <DirEntry 'OMFIT-2019-2-Monthly-Servicer-Report-Aug-202

In [55]:
initial_pool_info_dict = {'bal': {}, 'num_loans': {}, 'wa_fico': {}, 'wac': {}, 'wart': {}}

for i in range(len(files_dates)):
    print(files_dates[i][1])
    initial_pool = pd.read_excel(files_dates[i][1], usecols='B:F', skiprows=17, nrows=1, header=None, names=['bal', 'num_loans', 'wa_fico', 'wac', 'wart']).reset_index(drop=True)
#     initial_pool.columns = ['bal', 'num_loans', 'wa_fico', 'wac', 'wart']
    initial_pool.index = [files_dates[i][0]]
    print(initial_pool)
    for key, value in initial_pool.to_dict().items():
        initial_pool_info_dict[key].update(value)

<DirEntry '2019-10-15-1001257227.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2019-10-15  9.473742e+08     127037      629  26.91    48
<DirEntry '2019-11-14-1500125662.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2019-11-14  9.473742e+08     127037      629  26.91    48
<DirEntry '2019-12-16-1001259387.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2019-12-16  9.473742e+08     127037      629  26.91    48
<DirEntry '2020-01-14-1500126468.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2020-01-14  9.473742e+08     127037      629  26.91    48
<DirEntry '2020-02-14-1001260774.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2020-02-14  9.473742e+08     127037      629  26.91    48
<DirEntry '2020-03-16-1500128078.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2020-03-16  9.473742e+08     127037      629  26.91    48
<DirEntry '2020-04-14-1500128822.xlsx'>
                     bal

                     bal  num_loans  wa_fico    wac  wart
2023-06-14  9.473742e+08     127037      629  26.91    48
<DirEntry 'omfit-2019-2-monthly-servicer-report-june-2023-excel_investorversion.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2023-07-14  9.473742e+08     127037      629  26.91    48
<DirEntry 'omfit-2019-2-monthly-servicer-report-july-2023-excel_investorversion.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2023-08-14  9.473742e+08     127037      629  26.91    48
<DirEntry 'omfit-2019-2-monthly-servicer-report-august-2023-excel_investorversion.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2023-09-14  9.473742e+08     127037      629  26.91    48
<DirEntry 'omfit-2019-2-monthly-servicer-report-september-2023-excel_investorversion.xlsx'>
                     bal  num_loans  wa_fico    wac  wart
2023-10-16  9.473742e+08     127037      629  26.91    48


In [51]:
initial_pool_info_dict

{'bal': {Timestamp('2019-10-15 00:00:00'): 947374173.76,
  Timestamp('2019-11-14 00:00:00'): 947374173.76,
  Timestamp('2019-12-16 00:00:00'): 947374173.76,
  Timestamp('2020-01-14 00:00:00'): 947374173.76,
  Timestamp('2020-02-14 00:00:00'): 947374173.76,
  Timestamp('2020-03-16 00:00:00'): 947374173.76,
  Timestamp('2020-04-14 00:00:00'): 947374173.76,
  Timestamp('2020-05-14 00:00:00'): 947374173.76,
  Timestamp('2020-06-15 00:00:00'): 947374173.76,
  Timestamp('2020-07-14 00:00:00'): 947374173.76,
  Timestamp('2020-08-14 00:00:00'): 947374173.76,
  Timestamp('2020-09-14 00:00:00'): 947374173.76,
  Timestamp('2020-10-14 00:00:00'): 947374173.76,
  Timestamp('2020-11-16 00:00:00'): 947374173.76,
  Timestamp('2020-12-14 00:00:00'): 947374173.76,
  Timestamp('2021-01-14 00:00:00'): 947374173.76,
  Timestamp('2021-02-16 00:00:00'): 947374173.76,
  Timestamp('2021-03-15 00:00:00'): 947374173.76,
  Timestamp('2021-04-14 00:00:00'): 947374173.76,
  Timestamp('2021-05-14 00:00:00'): 9473741

In [13]:
pool_info_dict = {'wac': {}, 'wa_fico': {}, 'wart': {}, 'ending_balance': {}, 'num_loans': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    pool_info_df = pd.read_excel(files_dates[i][1], usecols='F', skiprows=55+row_adjustment, nrows=5, header=None, names=['end_of_period']).T.reset_index(drop=True)
    pool_info_df.columns = ['wac', 'wa_fico', 'wart', 'ending_balance', 'num_loans']
    pool_info_df.index = [files_dates[i][0]]
    for key, value in pool_info_df.to_dict().items():
        pool_info_dict[key].update(value)

In [14]:
pool_info_df = pd.DataFrame(pool_info_dict)

In [15]:
pool_info_df.sort_index(inplace=True)

In [16]:
pool_info_df

,wac,wa_fico,wart,ending_balance,num_loans
2019-10-15,26.9000,629.00,47.000000,9.474302e+08,126478.0
2019-11-14,26.8690,628.73,47.143849,9.474250e+08,125787.0
2019-12-16,26.8148,628.49,47.091538,9.474261e+08,124616.0
2020-01-14,26.7715,628.29,46.898766,9.474281e+08,124101.0
2020-02-14,26.7108,628.09,46.575797,9.474294e+08,123814.0
2020-03-16,26.6160,627.89,46.255265,9.474349e+08,123437.0
2020-04-14,26.5487,627.77,45.885982,9.474282e+08,123667.0
2020-05-14,26.1735,627.50,45.929313,9.947993e+08,128450.0
2020-06-15,25.9223,627.48,45.504034,9.947980e+08,128628.0
2020-07-14,26.0488,627.68,45.133056,9.947959e+08,128909.0


In [18]:
prin_coll_dict = {'total_prin_coll': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    prin_coll = pd.read_excel(files_dates[i][1], usecols='F', skiprows=77+row_adjustment, nrows=1, header=None, names=['total_prin_coll'])
    prin_coll.index = [files_dates[i][0]]
    for key, value in prin_coll.to_dict().items():
        prin_coll_dict[key].update(value)

In [19]:
prin_coll = pd.DataFrame(prin_coll_dict)

In [20]:
prin_coll.sort_index(inplace=True)

In [21]:
prin_coll

,total_prin_coll
2019-10-15,21238282.59
2019-11-14,23244260.58
2019-12-16,21917157.92
2020-01-14,21558623.72
2020-02-14,22119150.17
2020-03-16,23210406.20
2020-04-14,23284459.01
2020-05-14,22080028.05
2020-06-15,25669886.34
2020-07-14,27359920.41


In [22]:
int_coll_dict = {'total_int_coll': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    int_coll = pd.read_excel(files_dates[i][1], usecols='F', skiprows=82+row_adjustment, nrows=1, header=None, names=['total_int_coll'])
    int_coll.index = [files_dates[i][0]]
    for key, value in int_coll.to_dict().items():
        int_coll_dict[key].update(value)

In [23]:
int_coll = pd.DataFrame(int_coll_dict)

In [24]:
int_coll.sort_index(inplace=True)

In [25]:
int_coll

,total_int_coll
2019-10-15,20422691.08
2019-11-14,20285276.13
2019-12-16,20819435.76
2020-01-14,19454255.41
2020-02-14,21098722.08
2020-03-16,20099411.73
2020-04-14,18897837.31
2020-05-14,19344946.61
2020-06-15,20537830.07
2020-07-14,20736804.90


In [27]:
wf_dict = {}

counter = 0
for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
    else:
        row_adjustment = 0
    distributions = pd.read_excel(files_dates[i][1], usecols='F', skiprows=101+row_adjustment, nrows=19+row_adjustment, header=None, names=['distributions'])
    wf_steps = pd.read_excel(files_dates[i][1], usecols='A', skiprows=101+row_adjustment, nrows=19+row_adjustment, header=None, names=['wf_steps'])
    distributions.index = [wf_steps['wf_steps']]
    distributions = distributions.T
    distributions.index = [payment_dates[i]]
    if counter == 0:
        wf_dict = distributions.to_dict()
    else:
        for key, value in distributions.to_dict().items():
            try:
                wf_dict[key].update(value)
            except KeyError:
                pass
    counter += 1

In [28]:
wf_df = pd.DataFrame(wf_dict)

In [29]:
wf_df.sort_index(inplace=True)

In [30]:
wf_df

,Indenture Trustee/Owner Trustee/Note Registrar/Back-up Servicer Fees,Indeminfied Amounts,Back-Up Servicer Fee and Servicing Transition @ 0.025%,Servicing Transition Costs,Servicing Fee @ 3.50%,Class A Monthly Interest Amount,First Priority Principal Payment,Class B Monthly Interest Amount,Second Priority Principal Payment,Class C Monthly Interest Amount,...,Class D Monthly Interest Amount,Fourth Priority Principal Payment,Required Reserve Account Amount,Regular Principal Payment Amount,Additional Transaction Participant Fees,Additional Indeminified Amounts,Residual Released to the PDA,Residual Released to the Depositor,Total,NaN
2019-10-15,833.33,0.0,7236.89,0.0,0.0,1363429.87,0.0,207828.13,0.0,144472.4,...,264735.00,0.0,4500000.0,0.0,0.0,0.0,0.0,15571298.79,22059834.41,NaN
2019-11-14,833.33,0.0,19738.13,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,14859761.80,21855915.01,NaN
2019-12-16,833.33,0.0,19738.02,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,15302707.50,22298860.60,NaN
2020-01-14,833.33,0.0,19738.04,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,13835964.36,20832117.48,NaN
2020-02-14,833.33,0.0,19738.08,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,15156152.55,22152305.71,NaN
2020-03-16,833.33,0.0,19738.11,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,13969082.95,20965236.14,NaN
2020-04-14,833.33,0.0,19738.23,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,9421484.98,NaN,NaN
2020-05-14,833.33,0.0,19738.09,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,NaN,0.0,0.0,0.0,9455438.13,NaN,NaN
2020-06-15,833.33,0.0,20724.99,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,9902784.24,16899924.31,NaN
2020-07-14,833.33,0.0,20724.96,0.0,0.0,1704287.33,0.0,259785.17,0.0,180590.5,...,330918.75,0.0,4500000.0,0.0,0.0,0.0,0.0,10875342.03,17872482.07,NaN


In [36]:
rsv_acct_dict = {'reserve_acct_bal': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 4
    else:
        row_adjustment = 0
    rsv_acct = pd.read_excel(files_dates[i][1], usecols='F', skiprows=126+row_adjustment, nrows=1, header=None, names=['reserve_acct_bal'])
    rsv_acct.index = [files_dates[i][0]]
    for key, value in rsv_acct.to_dict().items():
        rsv_acct_dict[key].update(value)

In [37]:
rsv_acct_dict

{'reserve_acct_bal': {Timestamp('2019-10-15 00:00:00'): 4500000,
  Timestamp('2019-11-14 00:00:00'): 4500000,
  Timestamp('2019-12-16 00:00:00'): 4500000,
  Timestamp('2020-01-14 00:00:00'): 4500000,
  Timestamp('2020-02-14 00:00:00'): 4500000,
  Timestamp('2020-03-16 00:00:00'): 4500000,
  Timestamp('2020-04-14 00:00:00'): 4500000,
  Timestamp('2020-05-14 00:00:00'): 4500000,
  Timestamp('2020-06-15 00:00:00'): 4500000,
  Timestamp('2020-07-14 00:00:00'): 4500000,
  Timestamp('2020-08-14 00:00:00'): 4500000,
  Timestamp('2020-09-14 00:00:00'): 4500000,
  Timestamp('2020-10-14 00:00:00'): 4500000,
  Timestamp('2020-11-16 00:00:00'): 4500000,
  Timestamp('2020-12-14 00:00:00'): 4500000,
  Timestamp('2021-01-14 00:00:00'): 4500000,
  Timestamp('2021-02-16 00:00:00'): 4500000,
  Timestamp('2021-03-15 00:00:00'): 4500000,
  Timestamp('2021-04-14 00:00:00'): 4500000,
  Timestamp('2021-05-14 00:00:00'): 4500000,
  Timestamp('2021-06-14 00:00:00'): 4500000,
  Timestamp('2021-07-14 00:00:00'):

In [42]:
oc_dict = {'total_oc': {}, 'required_oc': {}, 'in_compliance': {}}

for i in range(len(files_dates)):
    if files_dates[i][0] <= pd.Timestamp('2021-08-16 00:00:00'):
        row_adjustment = 2
        if (files_dates[i][0] == pd.Timestamp('2020-04-14 00:00:00')) or (files_dates[i][0] == pd.Timestamp('2020-05-14 00:00:00')):
            row_adjustment += 2
    else:
        row_adjustment = 0
    oc = pd.read_excel(files_dates[i][1], usecols='F', skiprows=151+row_adjustment, nrows=3, header=None, names=['oc'])
    oc.index = ['total_oc', 'required_oc', 'in_compliance']
    oc = oc.T
    oc.index = [files_dates[i][0]]
    for key, value in oc.to_dict().items():
        oc_dict[key].update(value)

In [43]:
oc_dict

{'total_oc': {Timestamp('2019-10-15 00:00:00'): 47430162.18,
  Timestamp('2019-11-14 00:00:00'): 47424989.110000014,
  Timestamp('2019-12-16 00:00:00'): 47426092.05999994,
  Timestamp('2020-01-14 00:00:00'): 47428054.870000005,
  Timestamp('2020-02-14 00:00:00'): 47429412.48000002,
  Timestamp('2020-03-16 00:00:00'): 47434890.57000005,
  Timestamp('2020-04-14 00:00:00'): 47428211.70000005,
  Timestamp('2020-05-14 00:00:00'): 94799319.07000005,
  Timestamp('2020-06-15 00:00:00'): 94798041.16999996,
  Timestamp('2020-07-14 00:00:00'): 94795894.36000001,
  Timestamp('2020-08-14 00:00:00'): 94794517.85000002,
  Timestamp('2020-09-14 00:00:00'): 94798725.69000006,
  Timestamp('2020-10-14 00:00:00'): 94793667.05999994,
  Timestamp('2020-11-16 00:00:00'): 94795264.23000002,
  Timestamp('2020-12-14 00:00:00'): 94794532.32000005,
  Timestamp('2021-01-14 00:00:00'): 94794863.28999996,
  Timestamp('2021-02-16 00:00:00'): 94798179.24000001,
  Timestamp('2021-03-15 00:00:00'): 94797072.21000004,
  